In [2]:
import cv2
import numpy as np
import os

def extract_frames(video_path):
    """
    Extract frames from a video and return them as a list.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def resize_frame(frame, max_size=1200):
    """
    Resize the frame to make sure its dimensions are within the allowed limits.
    """
    h, w = frame.shape[:2]
    if max(h, w) > max_size:
        scaling_factor = max_size / max(h, w)
        new_size = (int(w * scaling_factor), int(h * scaling_factor))
        return cv2.resize(frame, new_size)
    return frame

def create_panorama(frames):
    """
    Create a panorama from the list of frames, resizing each frame to avoid excessive dimensions.
    """
    # Initialize the ORB detector
    orb = cv2.ORB_create()

    # Initialize a brute-force matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Resize the first frame if necessary
    panorama = resize_frame(frames[0])

    for i in range(1, len(frames)):
        # Resize the current frame if necessary
        frame_resized = resize_frame(frames[i])

        # Find keypoints and descriptors
        kp1, des1 = orb.detectAndCompute(panorama, None)
        kp2, des2 = orb.detectAndCompute(frame_resized, None)

        # Match descriptors between consecutive frames
        matches = bf.match(des1, des2)
        matches = sorted(matches, key=lambda x: x.distance)

        # Prepare points for homography estimation
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        # Find homography
        H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # Warp the second frame to the first frame
        panorama = cv2.warpPerspective(panorama, H, (panorama.shape[1] + frame_resized.shape[1], panorama.shape[0]))
        panorama[0:frame_resized.shape[0], 0:frame_resized.shape[1]] = frame_resized

    return panorama

def main():
    video_path = "video.mp4"  # Path to the input video
    frames = extract_frames(video_path)

    # Create a panorama
    panorama = create_panorama(frames)

    # Save the final panorama
    output_path = "output.jpg"
    cv2.imwrite(output_path, panorama)
    print(f"Panorama saved at {output_path}")

if __name__ == "__main__":
    main()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:1813: error: (-215:Assertion failed) dst.cols < SHRT_MAX && dst.rows < SHRT_MAX && src.cols < SHRT_MAX && src.rows < SHRT_MAX in function 'cv::remap'
